### accident_count

In [ ]:
!pip install geopy

In [ ]:
from geopy.distance import geodesic

def accident_count(lat, lon, radi, data, idx=False):
  """
  input: origin lat lon, radi(km), data(accidents record)
  output: following point count
  optional: (idx of point, count)
  """
  
  # #data import
  # crawled_df = pd.read_csv('2021_01_12_03_44_final.csv')
  # crawled_df['geometry'] = list(zip(crawled_df.iloc[:,-2],crawled_df.iloc[:,-1]))
  crawled_df = data

  #apply
  bool_mask = crawled_df['geometry'].apply(lambda x: geodesic((lat, lon),x).km <= radi)
  if idx:
    return crawled_df[bool_mask].index, sum(bool_mask)
  else:
    return sum(bool_mask)

# accident_count(crawled_df['geometry'].loc[111][0],crawled_df['geometry'].loc[111][1],0.05,True)
# #(Int64Index([111, 246], dtype='int64'), 2)

###road_nearby

In [ ]:
#interrial functions to use
from geopy.distance import great_circle as distance
from geopy.point import Point as Point
from math import sin, cos, atan2, sqrt, degrees, radians, pi


def midpoint(a, b):
    a_lat, a_lon = radians(a.latitude), radians(a.longitude)
    b_lat, b_lon = radians(b.latitude), radians(b.longitude)
    delta_lon = b_lon - a_lon
    B_x = cos(b_lat) * cos(delta_lon)
    B_y = cos(b_lat) * sin(delta_lon)
    mid_lat = atan2(
        sin(a_lat) + sin(b_lat),
        sqrt(((cos(a_lat) + B_x) ** 2 + B_y ** 2))
    )
    mid_lon = a_lon + atan2(B_y, cos(a_lat) + B_x)
    # Normalise
    mid_lon = (mid_lon + 3 * pi) % (2 * pi) - pi

    return Point(latitude=degrees(mid_lat), longitude=degrees(mid_lon))

def get_line_midpoint(line):
    a = Point(line[0])
    b = Point(line[1])

    return midpoint(a,b)


def calculate_dist_to_line(line_a_lat, line_a_lng, line_b_lat, line_b_lng, point_object):
    a = Point(latitude=line_a_lat, longitude=line_a_lng)
    b = Point(latitude=line_b_lat, longitude=line_b_lng)
    dist = distance(midpoint(a, b), point_object)
    return dist


def get_min_distance_to_arr(arr_coords, point_object, unit='m'):
    min_dist = 999999
    line=[]
    for i, _ in enumerate(arr_coords):
        if i + 1 < len(arr_coords):
            dist = calculate_dist_to_line(
                line_a_lat=arr_coords[i][1],
                line_a_lng=arr_coords[i][0],
                line_b_lat=arr_coords[i + 1][1],
                line_b_lng=arr_coords[i + 1][0],
                point_object=point_object
            )
            if dist < min_dist:
                min_dist = dist
                line = [(arr_coords[i][1], arr_coords[i][0]), (arr_coords[i + 1][1], arr_coords[i + 1][0])]
        else:
            dist = calculate_dist_to_line(
                line_a_lat=arr_coords[i][1],
                line_a_lng=arr_coords[i][0],
                line_b_lat=arr_coords[0][1],
                line_b_lng=arr_coords[0][0],
                point_object=point_object
            )
            if dist < min_dist:
                min_dist = dist
                line = [(arr_coords[i][1], arr_coords[i][0]), (arr_coords[0][1], arr_coords[0][0])]

    if unit == 'm':
        return min_dist.m, line
    elif unit == 'km':
        return min_dist.km, line
    else:
        return min_dist, line


In [ ]:
#additional interrial function
def _multiline_to_arr_cords(mls):
  """
  input: multilinestring object
  output: arr_cords; lon, lat
  ex) multiline_to_arr_cords(road_gdf.loc[0].geometry)
  """
  arr_cords = []
  lon_lat = re.findall(r'\d+\.\d+',str(mls))
  for i in range(0,len(lon_lat),2):
    arr_cords.append((float(lon_lat[i]),float(lon_lat[i+1])))
  return arr_cords

#(lat, lon), (lon, lat) 자꾸 갈려서 나오네
def _shPoint_to_list(shpoint):
  c1, c2 = re.findall(r'\d+\.\d+',str(shpoint))
  if c1 < c2:
    lon, lat = float(c1), float(c2)
  else:
    lat, lon = float(c1), float(c2)
  return lat, lon

def _list_to_gpPoint(mylist):
  return Point(latitude=mylist[0], longitude=mylist[1])

def _shPoint_to_gpPoint(shpoint):
  mylist = _shPoint_to_list(shpoint)
  return Point(latitude=mylist[0], longitude=mylist[1])

In [ ]:
#from https://medium.com/analytics-vidhya/calculating-distances-from-points-to-polygon-borders-in-python-a-paris-example-3b597e1ea291

def road_nearby(lat, lon, radi, data):
  """
  input: origin lat lon, radi(km), data(shapely multilinestring)
  change in type: shapely(Euclidian) -> list -> geopy(Great Circle)
  #watchout for (lat lon), (lon lat)
  output: following roads
  
  #test_set
  lat, lon = (37.14875860564099, 127.0773701360968)
  radi = 1
  road_nearby(lat, lon, radi, road_gdf)
  """
  road_gdf = data
  dist_array = road_gdf.geometry.apply(lambda x: get_min_distance_to_arr(_multiline_to_arr_cords(x), _shPoint_to_gpPoint((lat, lon)), unit='m')[0]/1000)
  return road_gdf.loc[dist_array < radi,:]
  


In [ ]:
# #Euclidian distance 쓰려면

# from shapely.geometry import shape
# import fiona

# p111 = shape({'type': 'Point', 'coordinates': crawled_df.loc[111].geometry})
# p222 = shape({'type': 'Point', 'coordinates': crawled_df.loc[222].geometry})
# p333 = shape({'type': 'Point', 'coordinates': crawled_df.loc[333].geometry})

# road1 = road_gdf.loc[0].geometry
# road1.distance(p111)

127.1039010288019

###chaos1, width_nearby, cross_road_nearby

In [ ]:
def chaos1_nearby(lat, lon, radi, road_data, chaos1_data):
  """
  input: origin lat lon, radi(km), road_data(shapely multilinestring), chaos1_data
  output: following road's chaos1 by different times or lanes
  
  #test_set
  lat, lon = (37.14875860564099, 127.0773701360968)
  radi = 1
  chaos1_nearby(lat, lon, radi, road_gdf, chaos1_gdf)[11]
  #link_id 끝자리 01 02 순으로 나옴
  """
  roads = road_nearby(lat, lon, radi, road_data)
  result = roads.link_id.apply(lambda x: chaos1_data.loc[(chaos1_data.loc[:,'link_id']//100)==int(x),['time_range','work/weekend','chaos1']])
  return result

def width_nearby(lat, lon, radi, road_data):
  roads = road_nearby(lat, lon, radi, road_data)
  return roads.width

def cross_road_nearby(lat, lon, radi, road_data):
  """
  4	교차로 통로
  32	복합교차로
  64	로타리
  128	회전교차로

  output = # of cross roads
  """
  cross_roads = [4,32,64,128]
  roads = road_nearby(lat, lon, radi, road_data)
  result = roads.loc[roads.link_type.apply(lambda x: int(x) in cross_roads)]
  return len(result)